# Setup

Dependencies:
- System: python3
- Python: jupyter, numpy, matplotlib, scipy, jax, cvxpy

Example setup for a Ubuntu system (Mac users, maybe `brew` instead of `sudo apt`; Windows users, learn to love [WSL](https://docs.microsoft.com/en-us/windows/wsl/install-win10)):
```
/usr/bin/python3 -m pip install --upgrade pip
pip install --upgrade jupyterlab numpy matplotlib scipy jax cvxpy
jupyter lab  # from the directory of this notebook
```
Alternatively, view this notebook on [Google Colab](https://colab.research.google.com/github/StanfordASL/AA203-Examples/blob/master/Zermelo's%20Problem.ipynb).

In [ ]:
import matplotlib.pyplot as plt

import numpy as np

from scipy.interpolate import interp1d
from scipy.optimize import minimize, Bounds

# Zermelo's Ship

Zermelo's navigation problem is a classic optimal control problem proposed in 1931 by Ernst Zermelo. In it, we have a ship that must cross a flowing river from a point on one bank to a point on the opposite bank. The ship travels at a fixed forward velocity. We can control its heading, which is bounded.

<center><img src="img/zermelo.png" width="600"/></center>

Our version of this optimal control problem is formulated as
$$
\begin{aligned}
\operatorname*{minimize}_{x,u}\enspace
&\int_0^T u(t)^2 \,dt
\\
\operatorname{subject~to}\enspace
&\dot{x}(t) = \begin{pmatrix}v\cos(u(t)) + w(x(t)) \\ v\sin(u(t))\end{pmatrix},\ \forall t \in [0,T] \\
&x(0) = 0 \\
&x(T) = (r_1, r_2) \\
&x_2(t) \in [0, r_2],\ \forall t \in [0,T] \\
&|u(t)| \leq \bar{u},\ \forall t \in [0,T]
\end{aligned}
$$
where $x_1(t)$ is the position of the ship along the river bank, $x_2(t)$ is the position of the ship across the water, $v > 0$ is the fixed forward velocity of the ship, $r = (r_1, r_2)$ is the position we wish to reach, and $w(x(t))$ represents the effect of the flowing water on our ship. Overall, we want to minimize the control effort used to reach our destination.

In the code below, we apply a few direct methods from class to solve this problem.

## Direct multiple shooting

In [ ]:
def solve_multiple_shooting(N=20, eps=1e-3, verbose=True,
                            t_init=None, x_init=None, u_init=None):
    """Solve the Zermelo navigation problem with direct multiple shooting."""
    # Constants
    n = 2           # state dimension
    m = 1           # control dimension
    T = 10.         # time horizon
    dt = T/N        # time step
    rx = 10.        # distance to destination along river bank
    ry = 5.         # width of the river
    v = 1.          # ship forward speed
    w_max = 0.35    # maximum water flow speed
    u_max = 0.75    # control magnitude bound
    x0 = np.array([0., 0.])
    xT = np.array([rx, ry])

    # Define how to map between `(x, u)` trajectories and decision variable `z`
    get_xu = lambda z: (z[:n*(N + 1)].reshape((N + 1, n)), z[-N:])
    get_z = lambda x, u: np.concatenate([x.ravel(), u])


    def dynamics(x, u):
        """Evaluate the continuous-time dynamics of Zermelo's ship."""
        # Water flow (quadratic profile)
        y = x[..., 1]
        w = (4*w_max/ry**2) * y * (ry - y)

        # Compute state derivative (vectorized)
        dx = np.column_stack([v*np.cos(u) + w, 
                              v*np.sin(u)])
        return dx


    def dynamics_discretized(x, u, dt=dt):
        """Evaluate the discretized dynamics of Zermelo's ship."""
        x_next = x + dt*dynamics(x, u)
        return x_next


    def cost(z, dt=dt):
        """Evaluate the cost function of the discretized problem."""
        _, u = get_xu(z)
        J = dt*np.sum(u**2)
        return J


    def constraints(z):
        """Evaluate the equality constraints of the discretized problem."""
        x, u = get_xu(z)
        h = np.concatenate([
            np.ravel(x[1:] - dynamics_discretized(x[:-1], u)),  # dynamics
            x[0] - x0,                                          # initial
            x[-1] - xT,                                         # terminal
        ])
        return h

    # Define the inequality constraints of the discretized problem
    x_lb = np.broadcast_to([-np.inf, 0. - eps], (N + 1, n))
    x_ub = np.broadcast_to([np.inf, ry + eps], (N + 1, n))
    u_lb = np.broadcast_to(-u_max - eps, (N,))
    u_ub = np.broadcast_to(u_max + eps, (N,))

    t = dt*np.arange(N + 1)

    if x_init is None:
        # Initialize with straight line from `x0` to `xT`
        s = np.linspace(0, 1, N + 1)
        x_init = x0 + (xT - x0)*s.reshape([-1, 1])
    elif t_init is not None:
        x_init = interp1d(t_init, x_init, axis=0)(t)
    else:
        raise ValueError('Argument `t_init` must be provided if `x_init` is '
                         'not `None`.')

    if u_init is None:
        # Initialize with some positive heading
        u_init = np.full(N, 0.5*u_max)
    elif t_init is not None:
        u_init = interp1d(t_init[:-1], u_init, axis=0)(t[:-1])
    else:
        raise ValueError('Argument `t_init` must be provided if `x_init` is '
                         'not `None`.')

    z_lb = get_z(x_lb, u_lb)
    z_ub = get_z(x_ub, u_ub)
    z_init = get_z(x_init, u_init)
    result = minimize(
        cost, 
        z_init, 
        bounds=Bounds(z_lb, z_ub), 
        constraints={'type': 'eq', 'fun': constraints},
        options={'maxiter': 1000}
    )
    if verbose:
        print(result)
    z = result.x
    J = cost(z)
    x, u = get_xu(z)
    return t, x, u, J

In [ ]:
N = 30
eps = 1.
warm_start = False

if warm_start:
    t, x, u, J = solve_multiple_shooting(N, eps)
    fig, ax = plt.subplots(1, 2, figsize=(15, 5), dpi=100)
    ax[0].set_title('Optimal trajectory (slackened)')
    ax[0].plot(x[:, 0], x[:, 1])
    ax[0].set_xlabel(r'$x_1$')
    ax[0].set_ylabel(r'$x_2$')
    ax[1].set_title('Optimal control (slackened), $J(u) = {:.2f}$'.format(J))
    ax[1].plot(t[:-1], u)
    ax[1].set_xlabel(r'$t$')
    ax[1].set_ylabel(r'$u$')
else:
    t, x, u = None, None, None
t, x, u, J = solve_multiple_shooting(N, 0., t_init=t, x_init=x, u_init=u)
fig, ax = plt.subplots(1, 2, figsize=(15, 5), dpi=100)
ax[0].set_title('Optimal trajectory')
ax[0].plot(x[:, 0], x[:, 1])
ax[0].set_xlabel(r'$x_1$')
ax[0].set_ylabel(r'$x_2$')
ax[1].set_title('Optimal control, $J(u) = {:.2f}$'.format(J))
ax[1].plot(t[:-1], u)
ax[1].set_xlabel(r'$t$')
ax[1].set_ylabel(r'$u$')
plt.show()